# How to use `locals`
`locals` is a pure Python package that ingests JWST Wide-Field Slitless Spectroscopy data and returns a source catalog of all the low-mass stars in the field along with their calculated fundamental and secondary parameters.

### Requirements
- pip install SEDkit
- pip install bokeh

In [63]:
# Imports
from locals import source, catalog
from SEDkit import sed, spectrum, synphot
import astropy.units as q
import astropy.table as at
import numpy as np
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

## Making an SED for a single source (no WFSS spectra)

For each spectrum in our WFSS data products, a `Source` object is created. `locals` then searches Vizier to find ancillary photometry, parallaxes, and spectral types. Then it assembles an SED from the available data and calculates fundamental and atmospheric parameters.

Let's pretend one of the brown dwarfs in our field is [LHS2924](http://simbad.u-strasbg.fr/simbad/sim-id?Ident=LHS2924&NbIdent=1&Radius=2&Radius.unit=arcmin&submit=submit+id). Here is what `locals` is doing for each source under the hood.

In [64]:
# Coordinates of LHS 2924 (Usually taken from the WFSS source_list)
ra = 217.180137*q.deg
dec = 33.177539*q.deg

# Make the source object
src = source.Source(ra=ra, dec=dec, name='LHS 2924', spectral_type='L3.5')

# Find some photometry
src.find_SDSS()
src.find_2MASS()
src.find_WISE()
src.find_PanSTARRS()

# Find a distance from Gaia DR2
src.find_Gaia()

# Take a look
print('\n',src.photometry,'\n')

Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
Setting radius to (<Quantity 0.10127179992838364 solRad>, <Quantity 0.02008219196144978 solRad>)

   band           eff         app_magnitude ...      abs_flux_unc      bandpass
                  um                       ... erg / (Angstrom cm2 s)         
-------- ------------------- ------------- ... ---------------------- --------
  SDSS.u 0.35565239687948447 21.9449996948 ... 1.2549111006613915e-18   SDSS.u
  SDSS.g  0.4702495279331733 20.9559993744 ...  8.182773370880929e-19   SDSS.g
   PS1.g 0.48491168757054565 20.5669994354 ... 1.3484427632650426e-18    PS1.g
  SDSS.r  0.6175578881059804 18.5820007324 ...  9.186342638736616e-19   SDSS.r
   PS1.r  0.6201185707505671 18.4883995056 ...   6.60097046064012e-19    PS1.r
  Gaia.G  0.6597013110557894       16.6299 ...                    0.0   Gaia.G
  SDSS.i  0.7489976846298569 16.2070007324 ...  2.544563527396815e-18   SDSS.i
   PS1.i  0.7534955578987176 16.2096004486 ...  1.99242

Now we can calculate the fundamental parameters and plot the results.

In [65]:
src.results

param,value,unc,units
object,object,object,object
name,LHS 2924,--,--
age,6.0,4.0,Gyr
distance,10.99,0.02,pc
parallax,90.9962,0.12710000574588776,mas
radius,0.10127179992838364,0.02008219196144978,solRad
SpT,L3.5,--,--
spectral_type,73.5,0.5,--
membership,--,--,--
fbol,6.66e-11,7.11e-13,erg / (cm2 s)


In [66]:
fig = src.plot()

## Make SEDs for all sources in the WFSS field
Now that we see the output of a single source, we can run this on the entire `SourceCatalog` generated from the WFSS output. The difference here is that a WFSS spectrum will be included in each `Source`.

In [88]:
# Path to the JWST pipeline output directory
cat_path = resource_filename('locals', 'data/wfss/')

# Initialize empty source catalog
cat = catalog.SourceCatalog(cat_path)

Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)


Let's see the source list of all objects in this catalog.

In [89]:
cat.source_list

id,xcentroid,ycentroid,ra_icrs_centroid,dec_icrs_centroid,xmin,xmax,ymin,ymax,abmag,abmag_error,sky_bbox_ll,sky_bbox_ur,sky_bbox_lr,sky_bbox_ul,icrs_centroid
,pix,pix,,,pix,pix,pix,pix,,,"deg,deg","deg,deg","deg,deg","deg,deg","deg,deg"
int64,float64,float64,object,object,float64,float64,float64,float64,int64,float64,object,object,object,object,object
10,1023.89938168,1023.19367348,-3.4283850489866588e-06,-2.694720849093827e-06,940.0,1107.0,943.0,1106.0,9,0.0816191841269,"0.000729834375136,-0.000694686445181","359.999270901,0.000711962615737","0.000729834375136,-0.000694686445181","359.999270901,0.000711962615737","89.7455,-29.05744"
13,1023.89938168,1023.19367348,-3.4283850489866588e-06,-2.694720849093827e-06,940.0,1107.0,943.0,1106.0,9,0.0816191841269,"0.000729834375136,-0.000694686445181","359.999270901,0.000711962615737","0.000729834375136,-0.000694686445181","359.999270901,0.000711962615737","217.180137,33.177539"


All the sources are stored as `Source` instances within the `SourceCatalog.sources` attribute. 

For example, LHS 2924 is part of this catalog so let's make sure we get the same results as when it is run individually.

In [90]:
# Get the source instance
lhs2924 = cat.sources[1]

# Calculate the results
print(lhs2924.results)

# Plot it
fig = lhs2924.plot()

Setting radius to (<Quantity 0.8786209573091851 solRad>, <Quantity 0.06782337214316517 solRad>)
    param           value                unc             units    
------------- ------------------ ------------------- -------------
         name          Source 13                  --            --
          age                6.0                 4.0           Gyr
     distance              10.99                0.02            pc
     parallax            90.9962 0.12710000574588776           mas
       radius 0.8786209573091851 0.06782337214316517        solRad
          SpT                 --                  --            --
spectral_type                 --                  --            --
   membership                 --                  --            --
         fbol           6.57e-11            4.37e-13 erg / (cm2 s)
         mbol             13.974               0.007            --
         Lbol            9.5e+29             7.2e+27       erg / s
     Lbol_sun               -3.6 

Hooray!

Let's take a look at the other one, 2MASS 0558-2903, an extreme subdwarf M7.

In [75]:
# Get the source instance
subdwarf = cat.sources[0]

# Calculate the results
print(subdwarf.results)

# Plot it
fig = subdwarf.plot()

Setting radius to (<Quantity 0.8786209573091851 solRad>, <Quantity 0.06782337214316517 solRad>)
    param           value                unc             units    
------------- ------------------ ------------------- -------------
         name          Source 10                  --            --
          age                6.0                 4.0           Gyr
     distance              79.19                0.68            pc
     parallax            12.6276 0.10779999941587448           mas
       radius 0.8786209573091851 0.06782337214316517        solRad
          SpT                 --                  --            --
spectral_type                 --                  --            --
   membership                 --                  --            --
         fbol           4.57e-12             1.4e-14 erg / (cm2 s)
         mbol             16.868               0.003            --
         Lbol           3.43e+30            5.98e+28       erg / s
     Lbol_sun              -3.05 

Fantastic!

Let's see a table of the results.

In [36]:
cat.results

name,age [Gyr],distance [pc],parallax [mas],radius [solRad],spectral_type [--],membership [--],fbol [erg / (cm2 s)],mbol [--],Lbol [erg / s],Lbol_sun [--],Mbol [--],logg [--],mass [solMass],Teff [K]
str31,str31,str31,str31,str31,str31,str31,str31,str31,str31,str31,str31,str31,str31,str31
Source 10,6.0 +/- 4.0,79.19 +/- 0.68,12.6276 +/- 0.10779999941587448,0.11 +/- 0.01,--,--,3.48e-12 +/- 1.89e-14,17.164 +/- 0.006,2.61e+30 +/- 4.7e+28,-3.17 +/- 0.008,12.671 +/- 0.011,4.5 +/- 0.07,0.8896913720017506 +/- 0.0,2812 +/- 128
Source 13,6.0 +/- 4.0,10.99 +/- 0.02,90.9962 +/- 0.12710000574588776,0.101 +/- 0.02,--,--,6.82e-11 +/- 4.53e-13,13.933 +/- 0.007,9.86e+29 +/- 7.47e+27,-3.59 +/- 0.003,13.728 +/- 0.007,4.5 +/- 0.07,0.8896913720017506 +/- 0.0,2300 +/- 227
